# Generate 2D Keypoints

In [1]:
pip install rtmlib -i https://pypi.org/simple

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install onnxruntime

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import os
import json
from rtmlib import Body, draw_skeleton

In [ ]:
# Define paths
#video_folder = os.path.join('..', '3dmotion', '20250326', 'camera')
#output_folder = os.path.join('..', 'data', )  # Relative to where script is run

print(f"Current working directory: {os.getcwd()}")
# Paths based on your setup
video_folder = '/pvc/scratch/3dmotion/20250326/camera'
output_folder = '/home/grienenb/move3d/data'

# Create output folder if it doesn't exist
# os.makedirs(output_folder, exist_ok=True)

# Initialize pose estimation model
body = Body(
    mode='balanced',
    backend='onnxruntime',
    device='cpu'
)

load C:\Users\liogr\.cache\rtmlib\hub\checkpoints\yolox_m_8xb8-300e_humanart-c2c7a14a.onnx with onnxruntime backend


Downloading: "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/onnx_sdk/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.zip" to C:\Users\liogr\.cache\rtmlib\hub\checkpoints\rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.zip
100%|██████████| 48.4M/48.4M [00:26<00:00, 1.93MB/s]


load C:\Users\liogr\.cache\rtmlib\hub\checkpoints\rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.onnx with onnxruntime backend


In [ ]:
# Loop through videos
for filename in os.listdir(video_folder):
    if filename.endswith('.MP4'):
        video_path = os.path.join(video_folder, filename)
        output_video_path = os.path.join(output_folder, f'keypoints_{filename}')
        output_json_path = os.path.join(output_folder, f'keypoints_{os.path.splitext(filename)[0]}.json')

        print(f"Processing: {filename}")

        cap = cv2.VideoCapture(video_path)

        # Setup video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                      int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_size = (width, height)

        print(f"  FPS: {fps}, Frame Size: {frame_size}")
        if fps == 0 or width == 0 or height == 0:
            print(f"Skipping {filename} due to invalid FPS or frame size.")
            cap.release()
            continue
        
        out = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

        all_keypoints = []
        all_scores = []

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            keypoints, scores = body(frame)
            all_keypoints.append(keypoints.tolist())
            all_scores.append(scores.tolist())

            frame = draw_skeleton(frame, keypoints, scores, kpt_thr=0.5)
            out.write(frame)

            frame_count += 1
            if frame_count % 200 == 0:
                print(f"  Processed {frame_count} frames...")

        cap.release()
        out.release()
        print(f"Saved video: {output_video_path}")

        if frame_count == 0:
            print(f"No frames were written to {output_video_path}") 
        else:
            print(f"Saved {frame_count} frames to {output_video_path}")

        # Save keypoints to JSON
        json_data = {
            "video": filename,
            "keypoints": all_keypoints,
            "scores": all_scores
        }

        with open(output_json_path, 'w') as f:
            json.dump(json_data, f)

        print(f"Saved keypoints JSON: {output_json_path}")

cv2.destroyAllWindows()


load C:\Users\liogr\.cache\rtmlib\hub\checkpoints\yolox_m_8xb8-300e_humanart-c2c7a14a.onnx with onnxruntime backend
load C:\Users\liogr\.cache\rtmlib\hub\checkpoints\rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.onnx with onnxruntime backend
Processing: C0110.MP4


KeyboardInterrupt: 